In [2]:
# 학습된 모델 불러다 다시 사용
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import cifar10

In [3]:
# 신경망 구조와 가중치를 저장하고 있는  파일을 읽어옴
cnn = tf.keras.models.load_model("/content/cnn1.h5")
cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 14, 14, 32)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 64)        18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 10, 10, 64)        36928     
                                                        

In [4]:
# 데이터 확보
(x_train,y_train),(x_test,y_test)= cifar10.load_data()
x_train = x_train.astype(np.float32)/255.0
x_test = x_test.astype(np.float32)/255.0

y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

170508288/170498071 [==============================] - 6s 0us/step


In [5]:
res = cnn.evaluate(x_test,y_test,verbose=2)
res

313/313 - 6s - loss: 0.6646 - accuracy: 0.7849 - 6s/epoch - 20ms/step


[0.6645742058753967, 0.7849000096321106]

In [6]:
# [0.664574146270752, 0.7849000096321106]

In [7]:
cnn.layers

In [9]:
cnn.layers[0].name

'conv2d'

In [ ]:
''